In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os 
import urllib

In [ ]:
import re

In [ ]:
urllib.request.urlretrieve('http://b.hiphotos.baidu.com/image/h%3D300/sign=92afee66fd36afc3110c39658318eb85/908fa0ec08fa513db777cf78376d55fbb3fbd9b3.jpg','sunflower.jpg')

![](sunflower.jpg)

In [ ]:
!ls &&pwd 
os.path.isfile("/Applications/Users/wangmiao/Playground/GH/IPython_training/basic/WiderKnowledge/")

In [ ]:
pattern=re.compile(r"\.\w{5}")
for root, dirs, files in os.walk("/Users/wangmiao/Playground/GH/IPython_training/basic/WiderKnowledge/"):
    for file in files:
        #if pattern.findall(file)[0] == '.ipynb':
        print( file)

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = "/Users/wangmiao/Playground/GH/IPython_training/basic/WiderKnowledge/"
[f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
pattern.findall("Decorators.ipynb")[0]

In [ ]:
class Card:
    def __init__(self):
        self.suit = ["Hearts","Diamonds","Clubs","Spades"]
        self.value = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
        self.cards = [] 

        for x in self.value:
            y = x + " of"
            for z in self.suit :
                aa = y + " " + z
                self.cards.append(aa)

    def __repr__(self): 
        return str(self.cards) 

cards1 = Card()

In [ ]:
cards1.cards

In [ ]:
ToEndofDays = 6388
#find today's date
import datetime
from datetime import date
EOD = "2037-01-04"
EOD = datetime.date(2037, 1, 4) 
print (EOD)
#d=datetime.datetime.today()
d=datetime.date.today()
# td = str(d(0-9))
# print (td)
delta = EOD - d
print ("Today is", d)
#print (ToEndofDays) 
#print ("I have until", EOD, "when I die"")
print(ToEndofDays, "days until the end!")
print (delta)

In [ ]:
s = "http://asaa.com"

In [ ]:
import requests, bs4

url = 'https://www.basketball-reference.com/players/a/abrinal01.html'
res = requests.get(url)
res.raise_for_status()

soup = bs4.BeautifulSoup(res.text, 'html.parser')
elems = soup.select('#per_game')

table = soup.find("table", { "id" : "per_game" })
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    
    print(row)

In [ ]:
import requests, bs4

url = 'https://www.basketball-reference.com/players/a/abrinal01.html'
res = requests.get(url)
res.raise_for_status()

soup = bs4.BeautifulSoup(res.text, 'html.parser')
elems = soup.select('#per_game')

table = soup.find("table", { "id" : "per_game" })
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td') + tr.find_all('th')
    row = [i.text for i in td]
    print(row)

In [ ]:
def througput():
    print("---------------menghitung througput Tower-------------------")
    print ("====Plih band====")

    print("[1]. 5 MHz, 25 RB")
    print("[2]. 10 MHz, 50 RB")
    print("[3]. 3 MHz, 75 RB")
    print("[4].  MHz, 100 RB")
    band =  int(input ("masukan pilihan (RB bandwith) : "))

    print ("====Plih Modulasi====")
    print("[1]. 64QAM")
    print("[2]. 16QAM")
    print("[3]. QPSK")
    mod =  int(input ("masukan pilihan (bits/symbol) : "))


    print ("====Plih Antena====")
    print("[1]. SISO")
    print("[2]. 2x2 MIMO")
    print("[3]. 4x4 MIMO")
    anten = int(input ("masukan pilihan (Pengali Antena: "))
    
    bandchoice = ["5 MHz, 25 RB", "10 MHz, 50 RB", "3 MHz, 75 RB", "MHz, 100 RB"]
    modchoice = ["64QAM", "16QAM", "QPSK"]
    antenchoice = ["SISO", "2x2 MIMO", "4x4 MIMO"]


    return ("band = {} mod = {}  anten= {}".format(bandchoice[band-1], modchoice[mod-1], antenchoice[anten-1]))

In [ ]:
througput()

In [ ]:
!cat xx.txt

# how-to-check-to-see-if-a-certain-line-is-found
https://stackoverflow.com/questions/57078822/how-to-check-to-see-if-a-certain-line-is-found-before-a-certain-point-in-a-txt-f

In [ ]:
find = 'Is this found'   

with open('xx.txt') as old_file:
  
    lines = old_file.readlines()
    print(lines[2])
    print(lines)
    for line in old_file:
        print(line)
        if line.startswith("This"):
            print("line")
    """
    with open(endfile1, "w") as new_file:
        for num, line in enumerate(lines,1):
            #if line "This is the" in line:
                line_base = num
        for line in lines:
            if not find in line.range(1:num):
                if line.startswith("This is the"):
                    line = newbasecase + line 
    """

In [ ]:
with open('test.txt') as f:
    lines = f.read().split('\n')
left = [line.split('=')[0].strip() for line in lines]
right = [line.split('=')[1].strip() for line in lines]
print(left)
print(right)



# Extrac data from xml

In [ ]:
from xml.etree import cElementTree as ET
xmlstr = """<poll title="User Suggested Number of Players" totalvotes="0" name="suggested_numplayers">
<results numplayers="3+"> 
</results></poll>
"""
root = ET.fromstring(xmlstr)
levels= root.findall('poll')
for level in levels:
    totalvotes = level.find('totalvotes').text
    print('totalvotes', totalvotes)

In [ ]:
xmlstr
a, b , c= xmlstr.partition("totalvotes=")
c.split()[0]

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(xmlstr) 
soup.find('poll').get('totalvotes')
for poll in soup.find_all('poll'):
    print (poll.get('totalvotes'))

In [ ]:
source = '''<root>
<level>
  <name>Matthias</name>
  <age>23</age>
  <gender>Male</gender>
</level>
<level>
  <name>Foo</name>
  <age>24</age>
  <gender>Male</gender>
</level>
<level>
  <name>Bar</name>
  <age>25</age>
  <gender>Male</gender>
</level>
</root>'''

root = ET.fromstring(source)
levels = root.findall('.//level')
for level in levels:
    name = level.find('name').text
    age = level.find('age').text
    print(name, age)

In [ ]:
largerList = [10,10,10,10,10,0,10,10,10,10,15,15,15,15,15,10,10,0,10,10,12,12,12,0]

sublist= [largerList[0]]
previous = largerList[0]
for item in largerList:
    #item 
    if item != previous:
        sublist.append(item)
        #sublist
        previous = item
sublist==[10,0,10,15,10,0,10,12,0]

In [ ]:
from itertools import groupby
LargerList = [10,10,10,10,10,0,10,10,10,10,15,15,15,15,15,10,10,0,10,10,12,12,12,0]
sublists = [k for k, _ in groupby(LargerList)]
sublists

In [ ]:
import requests
from bs4 import BeautifulSoup
page = requests.get('https://finviz.com/forex_performance.ashx')
soup = BeautifulSoup(page.content, 'html.parser')
forex = soup.find_all("div", {"class": "content "})
print(forex)

# List file 

In [ ]:
from os import listdir
from os.path import isdir
#files = [file for file in listdir('../storage') if isdir(file)]

for file in listdir('/Users/wangmiao/Desktop/'):
    print(file)

In [ ]:
!echo "CONFIG_PATH = '/usr/local/emarking/config/config.ini'" >> test.py

In [ ]:
!cat test.py

In [ ]:
!sed -i "33iCONFIG_PATH = '/usr/local/emarking/config/config.ini'" test.py

In [ ]:
regex_ = r'^((?!~\$).)+\.(?:xlsx?|csv|txt)$'

In [ ]:
all_files = [f_path + filename for f_path, _, filenames in os.walk('./') \
             for filename in filenames if re.search(regex_, filename)]
